In [16]:
from vlmeval.config import supported_VLM
from vlmeval.vlm.qwen_vl import QwenVL
from vlmeval.smp.vlm import isimg

In [29]:
model = QwenVL("/cephfs/panwenbo/work/mmcot_assets/models/Qwen-VL", max_new_tokens=512)

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

/cephfs/panwenbo/work/VLMEvalKit/vlmeval/vlm/qwen_vl.py:18: UserWarning: Following kwargs received: {'max_new_tokens': 512}, will use as generation config. 
  warnings.warn(f"Following kwargs received: {self.kwargs}, will use as generation config. ")


In [33]:
model.interleave_generate(
    ["https://qianwen-res.oss-cn-beijing.aliyuncs.com/Qwen-VL/assets/demo.jpeg", "Q: What is in this image please describe in detail/ A:"],
)

' The image shows a woman sitting on the beach with her dog, who is giving her a high five. The dog is wearing a blue harness and has a brown coat. The woman is wearing a plaid shirt and is holding a black cup. The sun is setting in the background, and the waves are crashing on the beach.'

In [7]:
hasattr(model, "interleave_generate")

True

In [47]:
def cut_seq(seq, by="\n\n"):
    idx = seq.find(by)
    if idx == -1:
        return seq
    return seq[:idx]

In [82]:
prompt = """Q: 这盘由哪种材料制成？ [[IMG: ../data/multi_lingual/images/train/18394/image.png]]
 Choice: (A) 棉布 (B) 塑料
A: 让我们一步一步地思考。[[因此，答案（英文字母）是]]"""

import re


def model_predict(model, prompt):
    # Extract image
    pattern = re.compile(r"\[\[IMG:.*?\]\]")
    all_imgs = pattern.findall(prompt)

    input_list = []
    for img in all_imgs:
        idx = prompt.find(img)
        assert isimg(img[6:-2].strip())
        if idx > 0:
            input_list.append(prompt[:idx].strip())
            input_list.append(img[6:-2].strip())
            prompt = prompt[idx + len(img) :]
    input_list.append(prompt)

    # Extract stage two prompt (extractor)
    extractor = None
    if "[[" in input_list[-1]:
        last_input = input_list[-1]
        pattern = re.compile(r"\[\[.*?\]\]")
        extractor = pattern.findall(last_input)[0][2:-2]
        last_input = pattern.sub("", last_input)
        input_list[-1] = last_input

    output = model.interleave_generate(input_list)
    output = cut_seq(output)
    seq_ret = output
    if extractor:
        input_list.append(output)
        input_list.append(extractor)
        output = model.interleave_generate(input_list)
        output = cut_seq(output)
        seq_ret += "\n" + extractor + output
    return seq_ret


model_predict(model, prompt)

'首先, 食物需要被装在容器里。因此, 选项 (A) 棉布和选项 (B) 塑料都是可以用来装食物的容器。但是, 塑料是更好的选择, 因为它更坚固, 不易碎, 而且可以重复使用。因此, 我们可以得出结论: 塑料是更好的选择。所以, 我选择 (B) 塑料。\n因此，答案（英文字母）是“B”。'

In [49]:
output = "asdfg\n\nfds"
output = cut_seq(output)
output

'asdfg'

In [40]:
print(input_list)
print(extractor)

['Q: 华盛顿的首都是什么？', '../data/multi_lingual/images/train/7189/image.png', 'Choice: (A) 奥林匹亚 (B) 亨廷顿 (C) 西雅图 (D) 拉拉米\nA: 让我们一步一步地思考。 奥林匹亚是华盛顿的首都。 因此，答案 (A, B, C, D) 是 A.\n\nQ: 这盘由哪种材料制成？', '../data/multi_lingual/images/train/18394/image.png', 'Choice: (A) 棉布 (B) 塑料\nA: 让我们一步一步地思考。 看看盘子的图片。\n盘子是由塑料制成的。\n大多数板是由塑料或陶瓷制成的。如果您放下陶瓷板，它可能会破裂！幼儿的板通常是用塑料制成的。 因此，答案 (A, B) 是 B.\n\nQ: 这些生物中的哪个包含曾经是地衣的一部分的物质？', '../data/multi_lingual/images/test/23/image.png', '\n Choice: (A) 覆盆子 (B) 蘑菇\nA: 让我们一步一步地思考。']
因此，答案 (A, B) 是


In [35]:
model.interleave_generate(input_list)

' 阿拉斯加苔原生态系统的食物链。 熊和狐狸以兔子为食。 蚯蚓以土壤中的蚯蚓为食。 雪地上的植物和土壤中的微生物以地衣为食。 因此,答案 (A, B) 是 B.\n\nQ: 这些动物中哪一个不属于爬行动物？Picture 4: 1'